In [1]:
# The code was removed by Watson Studio for sharing.

# Week 3 Assignment: Part 3

### Visualising the neighbourhood clusterings of Toronto

In [2]:
#retrieve the table from before
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_tor=pd.read_html(url, na_values=['Not assigned'])[0]
df_tor=df_tor.dropna(thresh=2) 
df_tor.columns = ['PostalCode','Borough','Neighborhood']
df_tor['Neighborhood']=df_tor['Neighborhood'].fillna(df_tor['Borough'])
df_tor1=df_tor.groupby(by=['PostalCode','Borough']).agg(lambda x: ','.join(x))
df_tor1.reset_index(inplace=True)
df_tor1.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [3]:
my_file = project.get_file("Geospatial_Coordinates.csv")
my_file.seek(0)
lnglat=pd.read_csv(my_file)
lnglat.columns=['PostalCode',"Latitude",'Longitude']
df_toronto = pd.merge(df_tor1, lnglat, on='PostalCode')
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
df_toronto.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

### Now we can begin on visualising the data and showing clusters.

##### For the next part we are going to work with the Borough of Central Toronto only.

In [5]:
df_cen=df_toronto[df_toronto['Borough'] == 'Central Toronto'].reset_index(drop=True)
df_cen.drop(columns=['PostalCode'],inplace=True) #we no longer need this
df_cen.head()

,Borough,Neighborhood,Latitude,Longitude
0,Central Toronto,Lawrence Park,43.728020,-79.388790
1,Central Toronto,Davisville North,43.712751,-79.390197
2,Central Toronto,North Toronto West,43.715383,-79.405678
3,Central Toronto,Davisville,43.704324,-79.388790
4,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160


In [6]:
#need to seperate out the neighborhoods again
df_ct=df_cen.assign(**{'Neighborhood':df_cen['Neighborhood'].str.split(',')})
df_ct=df_ct.Neighborhood.apply(pd.Series) \
    .merge(df_ct, right_index = True, left_index = True) \
    .drop(['Neighborhood'], axis = 1) \
    .melt(id_vars = ['Borough','Latitude','Longitude'], value_name = 'Neighborhood') \
    .drop('variable', axis = 1) \
    .dropna()
df_ct.head()

,Borough,Latitude,Longitude,Neighborhood
0,Central Toronto,43.728020,-79.388790,Lawrence Park
1,Central Toronto,43.712751,-79.390197,Davisville North
2,Central Toronto,43.715383,-79.405678,North Toronto West
3,Central Toronto,43.704324,-79.388790,Davisville
4,Central Toronto,43.689574,-79.383160,Moore Park


#### Next, we import all the required programs we need to visualise and form clusters.

In [7]:
import numpy as np 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

#### Let's map Central Toronto.

In [8]:
address = 'Central Toronto,ON'

geolocator = Nominatim(user_agent="ctor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Central Toronto are 43.653963, -79.387207.


In [28]:
# create map of Central Toronto using latitude and longitude values
map_ctor = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_ct['Latitude'], df_ct['Longitude'], df_ct['Borough'], df_ct['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ctor)  
    
map_ctor

Map as image:
![Central_Toronto_map]('https://github.com/Rolly-O/Segmenting-and-Clustering-for-Toronto/blob/master/central_toronto%20map.png')

In [29]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: provided and hidden for safety
CLIENT_SECRET: provided and hidden for safety


#### We want the top 100 venues that are in the neighbourhoods of Central Toronto within a radius of 500 meters.

In [11]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now we run the above function on each neighborhood in Central Toronto and create a new dataframe.

In [12]:
centor_venues = getNearbyVenues(names=df_ct['Neighborhood'],
                                   latitudes=df_ct['Latitude'],
                                   longitudes=df_ct['Longitude']
                                  )

Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park
Deer Park
Roselawn
Forest Hill North
The Annex
Summerhill East
Forest Hill SE
Forest Hill West
North Midtown
Rathnelly
Yorkville
South Hill
Summerhill West


In [13]:
print(centor_venues.shape)
centor_venues.head()

(236, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Dim Sum Deluxe,43.726953,-79.394260,Dim Sum Restaurant
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park


### Next, we analyse each neighborhood.

In [14]:
# one hot encoding
centor_onehot = pd.get_dummies(centor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
centor_onehot['Neighborhood'] = centor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [centor_onehot.columns[-1]] + list(centor_onehot.columns[:-1])
manhattan_onehot = centor_onehot[fixed_columns]

centor_onehot.head()

,American Restaurant,BBQ Joint,Bagel Shop,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Costume Shop,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Farmers Market,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Home Service,Hotel,Indian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Mexican Restaurant,Miscellaneous Shop,Park,Pharmacy,Pizza Place,Playground,Pool,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,Lawrence Park
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Davisville North


In [15]:
centor_onehot.shape #what is the new dataframe size

(236, 68)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [16]:
centor_grouped = centor_onehot.groupby('Neighborhood').mean().reset_index()
centor_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Costume Shop,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Farmers Market,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Home Service,Hotel,Indian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Mexican Restaurant,Miscellaneous Shop,Park,Pharmacy,Pizza Place,Playground,Pool,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.000000,0.027027,0.000,0.027027,0.000000,0.00,0.054054,0.000000,0.000000,0.054054,0.000000,0.027027,0.027027,0.081081,0.00,0.027027,0.027027,0.027027,0.000,0.027027,0.000000,0.027027,0.027027,0.000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000,0.027027,0.027027,0.054054,0.027027,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.027027,0.054054,0.0,0.000000,0.000000,0.000000,0.027027,0.000000,0.081081,0.027027,0.000000,0.000000,0.000000,0.0,0.000000,0.054054,0.00,0.0,0.054054,0.027027,0.00,0.000000,0.000000,0.000000
1,Davisville North,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.00,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.125,0.125000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,Deer Park,0.066667,0.000000,0.066667,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.066667,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,0.0,0.000000,0.133333,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0,0.066667,0.066667,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.066667,0.000000
3,Forest Hill North,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.00,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.000000
4,Forest Hill SE,0.066667,0.000000,0.066667,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.066667,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,0.0,0.000000,0.133333,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0,0.066667,0.066667,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.066667,0.000000
5,Forest Hill West,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000

We're now going to print each neighborhood with the 3 most common venues. Once, we're happy it works we will sort in descending order and put into a pandas dataframe the top 10 venues.

In [17]:
num_top_venues = 3

for hood in centor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = centor_grouped[centor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
             venue  freq
0     Dessert Shop  0.08
1   Sandwich Place  0.08
2  Thai Restaurant  0.05


----Davisville North----
            venue  freq
0  Sandwich Place  0.12
1           Hotel  0.12
2   Grocery Store  0.12


----Deer Park----
                 venue  freq
0                  Pub  0.13
1          Coffee Shop  0.13
2  American Restaurant  0.07


----Forest Hill North----
                venue  freq
0       Jewelry Store  0.25
1               Trail  0.25
2  Mexican Restaurant  0.25


----Forest Hill SE----
                 venue  freq
0                  Pub  0.13
1          Coffee Shop  0.13
2  American Restaurant  0.07


----Forest Hill West----
                venue  freq
0       Jewelry Store  0.25
1               Trail  0.25
2  Mexican Restaurant  0.25


----Lawrence Park----
                venue  freq
0         Swim School  0.25
1  Dim Sum Restaurant  0.25
2            Bus Line  0.25


----Moore Park----
         venue  freq
0  Summer Camp   0.2
1  

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = centor_grouped['Neighborhood']

for ind in np.arange(centor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(centor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Sandwich Place,Thai Restaurant,Pizza Place,Sushi Restaurant,Café,Italian Restaurant,Coffee Shop,Indoor Play Area,Restaurant
1,Davisville North,Clothing Store,Hotel,Sandwich Place,Breakfast Spot,Gym,Grocery Store,Food & Drink Shop,Park,Yoga Studio,Flower Shop
2,Deer Park,Pub,Coffee Shop,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Liquor Store,Pizza Place,Restaurant,American Restaurant
3,Forest Hill North,Mexican Restaurant,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Farmers Market,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden
4,Forest Hill SE,Pub,Coffee Shop,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Liquor Store,Pizza Place,Restaurant,American Restaurant


### Now we can cluster!

##### We run k-means to cluster the neighborhood into 5 clusters.

In [20]:
# set number of clusters
kclusters = 5

centor_grouped_clustering = centor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(centor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 2, 1, 2, 4, 3, 1, 1], dtype=int32)

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

centor_merged = df_ct

# merge centor_grouped with cf dataframe to add latitude/longitude for each neighborhood
centor_merged = centor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

centor_merged.head() # check the last columns!

,Borough,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,43.728020,-79.388790,Lawrence Park,4,Bus Line,Park,Swim School,Dim Sum Restaurant,Gourmet Shop,Farmers Market,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden
1,Central Toronto,43.712751,-79.390197,Davisville North,1,Clothing Store,Hotel,Sandwich Place,Breakfast Spot,Gym,Grocery Store,Food & Drink Shop,Park,Yoga Studio,Flower Shop
2,Central Toronto,43.715383,-79.405678,North Toronto West,1,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Spa,Mexican Restaurant,Miscellaneous Shop,Park,Diner,Dessert Shop
3,Central Toronto,43.704324,-79.388790,Davisville,1,Dessert Shop,Sandwich Place,Thai Restaurant,Pizza Place,Sushi Restaurant,Café,Italian Restaurant,Coffee Shop,Indoor Play Area,Restaurant
4,Central Toronto,43.689574,-79.383160,Moore Park,3,Summer Camp,Tennis Court,Restaurant,Playground,Trail,Gym / Fitness Center,Gym,Grocery Store,Greek Restaurant,Gourmet Shop


### Finally, we visualise the clusters.

In [27]:
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(centor_merged['Latitude'], centor_merged['Longitude'], centor_merged['Neighborhood'], centor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Map as image:
![clusters_map]('https://github.com/Rolly-O/Segmenting-and-Clustering-for-Toronto/blob/master/clustersmap.png')